In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


c:\Users\hp\OneDrive\Deep Learning\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
df = pd.read_csv('twitter_training.csv', header=None, names=['id', 'company', 'sentiment', 'text'])

df['text'] = df['text'].fillna('').str.lower().replace(r'[^a-z0-9\s]', '', regex=True)
df = df[df['text'].str.strip() != ''].reset_index(drop=True)

max_words = 10000
max_len = 100

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['text'])
X = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(X, maxlen=max_len)

le = LabelEncoder()
y = le.fit_transform(df['sentiment'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

model = Sequential([
    Embedding(input_dim=max_words+1, output_dim=128, input_length=max_len),
    GRU(64, dropout=0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

c:\Users\hp\OneDrive\Deep Learning\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
1843/1843 ━━━━━━━━━━━━━━━━━━━━ 226s 117ms/step - accuracy: 0.3025 - loss: -43.0079 - val_accuracy: 0.3012 - val_loss: -79.6155
Epoch 2/5
1843/1843 ━━━━━━━━━━━━━━━━━━━━ 169s 92ms/step - accuracy: 0.3025 - loss: -115.7388 - val_accuracy: 0.3012 - val_loss: -151.8239
Epoch 3/5
1843/1843 ━━━━━━━━━━━━━━━━━━━━ 121s 66ms/step - accuracy: 0.3025 - loss: -187.9599 - val_accuracy: 0.3012 - val_loss: -223.9492
Epoch 4/5
1843/1843 ━━━━━━━━━━━━━━━━━━━━ 123s 67ms/step - accuracy: 0.3025 - loss: -260.2047 - val_accuracy: 0.3012 - val_loss: -296.1590
Epoch 5/5
1843/1843 ━━━━━━━━━━━━━━━━━━━━ 124s 67ms/step - accuracy: 0.3025 - loss: -332.7210 - val_accuracy: 0.3012 - val_loss: -368.6398


In [ ]:

def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences([text.lower()]) 
    padded = pad_sequences(seq, maxlen=max_len)
    
    score = model.predict(padded, verbose=0)[0][0]
    sentiment = 'Positive' if score > 0.5 else 'Negative'
    conf = score if score > 0.5 else 1 - score
    return f"{sentiment} ({conf:.1%})"
samples = ["I love this!", "This is terrible.", "It was okay."]
for s in samples:
    print(f"'{s}' -> {predict_sentiment(s)}")

'I love this!' -> Positive (100.0%)
'This is terrible.' -> Positive (100.0%)
'It was okay.' -> Positive (100.0%)
